In [ ]:
import Oncoder
import torch
import pandas as pd
from Oncoder import Autoencoder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Training Oncoder to learn a new methylation atlas based on simulation data and deconvoluting the simulation data.

In [ ]:
refdata = 'data/reference_data.tsv'
train_x,train_y = Oncoder.generate_simulated_data(refdata,prior=[0.8,0.2],samplenum=1000,random_state=1)
test_x,test_y = Oncoder.generate_simulated_data(refdata,prior=[0.8,0.2],samplenum=100,random_state=1)

In [ ]:
model=Oncoder.train_Oncoder(train_x,train_y,refdata,model_name='Oncoder_test',batch_size=128,epochs=256)

In [ ]:
pred_y,methyatlas = Oncoder.predict(test_x,model)

In [ ]:
pred_y

Loading Oncoder for predicting read data.

In [ ]:
model = torch.load('model/Oncoder.pth',map_location=device)

In [ ]:
Normal_data = pd.read_csv('data/normal_data.tsv',sep='\t',index_col=0,header=0).T.values
HCC_data = pd.read_csv('data/HCC_data.tsv',sep='\t',index_col=0,header=0).T.values

In [ ]:
pred_y,methyatlas = Oncoder.predict(Normal_data,model)

In [ ]:
pred_y

Loading Oncoder for 9 CpG version to distinguish normal and HCC

In [ ]:
model = torch.load('model/Oncoder_9CpG.pth',map_location=device)

In [ ]:
n=pd.read_csv("data/HCC_9CpG.tsv",index_col=0,header=0,sep='\t')

In [ ]:
pred_y0,_=Oncoder.predict(n.filter(like='noHCC').T.values,model)
pred_y1,_=Oncoder.predict(n.filter(regex='^HCC(?!.*noHCC)').T.values,model)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
sns.set(style='white')

In [ ]:
data1 = pred_y0[:,1]
data2 = pred_y1[:,1]
_, p_value1 = stats.ranksums(data1, data2)
all_data = [data1, data2]

plt.figure(figsize=(8, 6))
sns.boxplot(x='variable', y='value', data=pd.melt(pd.DataFrame(all_data).T), palette='Set2',width = 0.3,fliersize=3)
plt.text(0.5, 0.65, f'$p$ = {"{:.2e}".format(p_value1)}', fontsize=15, color='r', ha='center')
plt.xlabel(' ')
plt.ylabel('Predicted tumor fraction',fontsize =16)
plt.xticks(ticks=[0, 1], labels=['normal', 'HCC'],fontsize =16)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc,accuracy_score
y_true = np.concatenate([np.zeros(len(data1)),np.ones(len(data2))])
predictions = np.concatenate([data1, data2])
fpr, tpr, thresholds = roc_curve(y_true, predictions)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc="lower right")
plt.show()
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print("Optimal threshold:", optimal_threshold)
predicted_labels = (predictions >= optimal_threshold).astype(int)
accuracy = accuracy_score(y_true, predicted_labels)
print("Accuracy at optimal threshold:", accuracy)
optimal_sensitivity = tpr[optimal_idx]
optimal_specificity = 1 - fpr[optimal_idx]
print("Optimal sensitivity:", optimal_sensitivity)
print("Optimal specificity:", optimal_specificity)

Loading Oncoder to distinguish the patients with HCC and the patients with HCC & Cirrhosis

In [ ]:
model = torch.load("model/Oncoder.pth",map_location=device)

In [ ]:
HCC = pd.read_csv('data/HCC_data.tsv',sep='\t',index_col=0)
Cirrhosis = pd.read_csv('data/Cirrhosis_data.tsv',sep='\t',index_col=0)
HCC = HCC.T.values
Cirrhosis = Cirrhosis.T.values
pred_HCC,methyHCC = Oncoder.predict(HCC,model) 
pred_Cir,methyCir = Oncoder.predict(Cirrhosis,model) 

In [ ]:
data1 = pred_Cir[:,1]
data2 = pred_HCC[:,1]
_, p_value = stats.ranksums(data1, data2)
all_data = [data1, data2]

plt.figure(figsize=(8, 6))
sns.boxplot(x='variable', y='value', data=pd.melt(pd.DataFrame(all_data).T), palette='Set2',width = 0.3,fliersize=1)
plt.text(0.5, 0.3, f'$p$ = {p_value:.4f}', fontsize=15, color='r', ha='center')
plt.xlabel(' ')
plt.ylabel('Predicted tumor fraction',fontsize=16)
plt.xticks(ticks=[0, 1], labels=['Cirrhosis alone', 'Cirrhosis & HCC'],fontsize=16)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
y_true = np.concatenate([np.zeros(len(data1)),np.ones(len(data2))])
predictions = np.concatenate([data1, data2])
fpr, tpr, thresholds = roc_curve(y_true, predictions)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc="lower right")
plt.show()
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print("Optimal threshold:", optimal_threshold)
optimal_sensitivity = tpr[optimal_idx]
optimal_specificity = 1 - fpr[optimal_idx]
print("Optimal sensitivity:", optimal_sensitivity)
print("Optimal specificity:", optimal_specificity)

Loading Oncoder for 7 CpG version (trianed on simulation data) to detect tumor

In [ ]:
model = torch.load("model/Oncoder_7CpG_S.pth",map_location=device)

In [ ]:
xu_data = pd.read_csv('data/xu_data.tsv',sep='\t',index_col=0)
xu_data.columns = [i.split('.')[0] for i in xu_data.columns.tolist()]

In [ ]:
pred_y0,_=Oncoder.predict(xu_data.filter(like='0').T.values,model)
pred_y1,_=Oncoder.predict(xu_data.filter(like='1').T.values,model)
pred_y2,_=Oncoder.predict(xu_data.filter(like='2').T.values,model)
pred_y3,_=Oncoder.predict(xu_data.filter(like='3').T.values,model)
pred_y4,_=Oncoder.predict(xu_data.filter(like='4').T.values,model)

In [ ]:
data1 = pred_y0[:,1]
data2 = pred_y1[:,1]
data3 = pred_y2[:,1]
data4 = pred_y3[:,1]
data5 = pred_y4[:,1]
_, p_value1 = stats.ranksums(data1, data2)
_, p_value2 = stats.ranksums(data2, data3)
_, p_value3 = stats.ranksums(data3, data4)
_, p_value4 = stats.ranksums(data4, data5)
all_data = [data1, data2, data3, data4, data5]

plt.figure(figsize=(8, 6))
sns.boxplot(x='variable', y='value', data=pd.melt(pd.DataFrame(all_data).T), palette='Set2',width = 0.3,fliersize=1)
plt.text(0.5, 0.23, f'$p$ = {"{:.2e}".format(p_value1)}', fontsize=13, color='r', ha='center')
plt.text(1.5, 0.38, f'$p$ = {"{:.2e}".format(p_value2)}', fontsize=13, color='r', ha='center')
plt.text(2.5, 0.57, f'$p$ = {"{:.2e}".format(p_value3)}', fontsize=13, color='r', ha='center')
plt.text(3.5, 0.85, f'$p$ = {"{:.2e}".format(p_value4)}', fontsize=13, color='r', ha='center')
plt.ylim(-0.05,1.05)
plt.xlabel(' ')
plt.ylabel('Predicted tumor fraction',fontsize=16)
plt.xticks(ticks=[0, 1,2,3,4], labels=['normal', 'stage1','stage2','stage3','stage4'],fontsize=16)
plt.yticks(fontsize=14)
plt.show()

Loading Oncoder for 7 CpG version (trained on read data) to detect tumor

In [ ]:
model = torch.load("model/Oncoder_7CpG_R.pth",map_location=device)

In [ ]:
xu_data = pd.read_csv("data/test_xu_data.tsv",index_col=0,sep='\t')
xu_data.columns = [i.split('.')[0] for i in xu_data.columns.tolist()]

In [ ]:
pred_y0,_=Oncoder.predict(xu_data.filter(like='0').T.values,model)
pred_y1,_=Oncoder.predict(xu_data.filter(like='1').T.values,model)
pred_y2,_=Oncoder.predict(xu_data.filter(like='2').T.values,model)
pred_y3,_=Oncoder.predict(xu_data.filter(like='3').T.values,model)
pred_y4,_=Oncoder.predict(xu_data.filter(like='4').T.values,model)

In [ ]:
data1 = pred_y0[:,1]
data2 = pred_y1[:,1]
data3 = pred_y2[:,1]
data4 = pred_y3[:,1]
data5 = pred_y4[:,1]
_, p_value1 = stats.ranksums(data1, data2)
_, p_value2 = stats.ranksums(data2, data3)
_, p_value3 = stats.ranksums(data3, data4)
_, p_value4 = stats.ranksums(data4, data5)
all_data = [data1, data2, data3, data4, data5]

plt.figure(figsize=(8, 6))
sns.boxplot(x='variable', y='value', data=pd.melt(pd.DataFrame(all_data).T), palette='Set2',width = 0.3,fliersize=1)
plt.text(0.5, 0.23, f'$p$ = {"{:.2e}".format(p_value1)}', fontsize=13, color='r', ha='center')
plt.text(1.5, 0.31, f'$p$ = {"{:.2e}".format(p_value2)}', fontsize=13, color='r', ha='center')
plt.text(2.5, 0.34, f'$p$ = {"{:.2e}".format(p_value3)}', fontsize=13, color='r', ha='center')
plt.text(3.5, 0.36, f'$p$ = {"{:.2e}".format(p_value4)}', fontsize=13, color='r', ha='center')
plt.ylim(-0.05,1.05)
plt.xlabel(' ')
plt.ylabel('Predicted tumor fraction',fontsize=16)
plt.xticks(ticks=[0, 1,2,3,4], labels=['normal', 'stage1','stage2','stage3','stage4'],fontsize=16)
plt.yticks(fontsize=14)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc, accuracy_score
y_true = np.concatenate([np.zeros(len(data1)),np.ones(len(data2)),np.ones(len(data3)),np.ones(len(data4)),np.ones(len(data5))])
predictions = np.concatenate([data1, data2,data3,data4,data5])
fpr, tpr, thresholds = roc_curve(y_true, predictions)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Curve')
plt.legend(loc="lower right")
plt.show()
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print("Optimal threshold:", optimal_threshold)
predicted_labels = (predictions >= optimal_threshold).astype(int)
accuracy = accuracy_score(y_true, predicted_labels)
print("Accuracy at optimal threshold:", accuracy)
optimal_sensitivity = tpr[optimal_idx]
optimal_specificity = 1 - fpr[optimal_idx]
print("Optimal sensitivity:", optimal_sensitivity)
print("Optimal specificity:", optimal_specificity)

Changes in predicted tumor fractions before and after treatment were assessed in two groups: patients sensitive and resistant to abiraterone acetate.

In [ ]:
sensitive=pd.read_csv("data/sensitive_group.csv",header=0)
resistant=pd.read_csv("data/resistant_group.csv",header=0)

In [ ]:
from scipy.stats import wilcoxon

In [ ]:
y_before =sensitive.groupby('patientsid').first()['Oncoder_10'].tolist() #Oncoder_10 : the result of  Oncoder for PRAD version based on top 10% stable marker
y_after = sensitive.groupby('patientsid').last()['Oncoder_10'].tolist()   
x_before = 0.25  
x_after = 0.75
_, p_value = wilcoxon(y_before, y_after)

plt.figure(figsize=(8, 6)) 
  
for i in range(len(y_before)):  
    plt.scatter([x_before, x_after], [y_before[i], y_after[i]], label=f'Patient {i+1}',color='#009dff')  
    plt.plot([x_before, x_after], [y_before[i], y_after[i]], color='#009dff')  
  
plt.ylabel('Predicted tumor fraction',fontsize=16)  
plt.text(0.5, 0.6, f'$p$ = {p_value:.4f}', fontsize=15, color='r', ha='center')
plt.ylim(0,0.7)
plt.xlim(0.1,0.9)
plt.yticks(fontsize=14)   
plt.xticks([x_before, x_after], ['Before', 'After'],fontsize=16)  
plt.title("Treatment sensitive",fontsize=16)
plt.show()

In [ ]:
y_before =resistant.groupby('patientsid').first()['Oncoder_10'].tolist()  
y_after = resistant.groupby('patientsid').last()['Oncoder_10'].tolist()   
x_before = 0.25  
x_after = 0.75
_, p_value = wilcoxon(y_before, y_after)

plt.figure(figsize=(8, 6))
  
for i in range(len(y_before)):  
    plt.scatter([x_before, x_after], [y_before[i], y_after[i]], label=f'Patient {i+1}',color='#ffc4a1')  
    plt.plot([x_before, x_after], [y_before[i], y_after[i]], color='#ffc4a1')  
  
plt.ylabel('Predicted tumor fraction',fontsize=16)  
plt.ylim(0,0.7)
plt.xlim(0.1,0.9)
plt.text(0.5, 0.6, f'$p$ = {p_value:.4f}', fontsize=15, color='r', ha='center')
plt.yticks(fontsize=14)   
plt.xticks([x_before, x_after], ['Before', 'After'],fontsize=16)  
plt.title("Treatment resistant",fontsize=16)
plt.show()